In [ ]:
!pip install torch torchvision ultralytics

In [3]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

# Load the dataset
dataset = fouh.load_from_hub("Voxel51/VisDrone2019-DET",overwrite=True)

# Launch the App
session = fo.launch_app(dataset)

Loading dataset
Importing samples...
 100% |███████████████| 8629/8629 [490.7ms elapsed, 0s remaining, 17.6K samples/s]      


In [ ]:
import fiftyone.utils.random as four

classes = dataset.distinct("ground_truth.detections.label")

four.random_split(dataset, {"val": 0.15, "train": 0.85})


for split in ["val","train","test"]:
    view =  dataset.match_tags(split)
    view.export(
        export_dir="VisDrone_YOLO/",
        split=split,
        dataset_type=fo.types.YOLOv5Dataset,
        classes=classes
    )

 100% |███████████████| 2804/2804 [21.3s elapsed, 0s remaining, 180.4 samples/s]      
Directory 'VisDrone_YOLO/' already exists; export will be merged with existing files
 100% |███████████████| 8578/8578 [1.1m elapsed, 0s remaining, 153.2 samples/s]      
Directory 'VisDrone_YOLO/' already exists; export will be merged with existing files
   6% |/--------------|  101/1610 [1.2s elapsed, 18.3s remaining, 82.6 samples/s]     

In [1]:
from ultralytics import YOLO


# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov8n.pt')

# Display model information (optional)
model.info()

# Train the model on the  dataset for 1 epochs, set project name for experiment_name, name for the run name
model.train(
    data='VisDrone_YOLO/dataset.yaml',
    imgsz=640, 
    batch=1,
)

# Add predictions to our dataset
dataset.apply_model(model, label_field="YOLOv8")


YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs
New https://pypi.org/project/ultralytics/8.2.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.25 🚀 Python-3.9.18 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12010MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=VisDrone_YOLO/dataset.yaml, epochs=100, time=None, patience=100, batch=1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

train: Scanning /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/labels/train.cache... 8578 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8578/8578 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/images/train/0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/images/train/0000140_00118_d_0000002.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/images/train/9999945_00000_d_0000114.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/images/train/9999987_00000_d_0000049.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/images/train/9999998_00219_d_0000175.jpg: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/dan/Documents/cookbook/Detection/VisDrone_YOLO/labels/val.cache... 2804 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2804/2804 [00:00<?, ?it/s]

...

MLflow: logging run_id(7edd11d42efa4ef7b98147d28b39be97) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100     0.499G      1.982      2.686      1.095        115        640: 100%|██████████| 8578/8578 [04:07<00:00, 34.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1402/1402 [00:26<00:00, 52.54it/s]


                   all       2804     161413       0.49      0.115     0.0866     0.0474

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100     0.476G      1.921      2.117      1.066         10        640:  43%|████▎     | 3656/8578 [01:43<02:19, 35.28it/s]


KeyboardInterrupt: 